In [39]:
import torch

# FORCE GPU
if not torch.cuda.is_available():
    raise Exception("GPU not available. Please enable CUDA.")

device = torch.device("cuda")
print("Using device:", torch.cuda.get_device_name(0))


import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform (normalize CIFAR)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# Load CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

# LeNet Model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)   # changed input channel to 3
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))   # 32->28->14
        x = self.pool(torch.relu(self.conv2(x)))   # 14->10->5
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop (10 epochs)
for epoch in range(10):
    model.train()
    running_loss = 0.0
    
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/10], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training Finished")


Using device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1311.7113, Accuracy: 47.26%
Epoch [2/10], Loss: 1079.2195, Accuracy: 53.05%
Epoch [3/10], Loss: 994.9199, Accuracy: 54.40%
Epoch [4/10], Loss: 940.4686, Accuracy: 56.50%
Epoch [5/10], Loss: 894.0639, Accuracy: 57.98%
Epoch [6/10], Loss: 860.0616, Accuracy: 59.23%
Epoch [7/10], Loss: 825.8300, Accuracy: 59.22%
Epoch [8/10], Loss: 793.0323, Accuracy: 59.55%
Epoch [9/10], Loss: 766.4474, Accuracy: 59.56%
Epoch [10/10], Loss: 741.7228, Accuracy: 60.07%
Training Finished


In [1]:
import torch

# FORCE GPU
if not torch.cuda.is_available():
    raise Exception("GPU not available. Please enable CUDA.")

device = torch.device("cuda")
print("Using device:", torch.cuda.get_device_name(0))


import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform (normalize CIFAR)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# Load CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

Using device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Files already downloaded and verified
Files already downloaded and verified


In [6]:
def train_model(model, trainloader, testloader, epochs=10, lr=0.001):

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    total_batches = len(trainloader)

    for epoch in range(epochs):
        model.train()
        correct_train = 0
        total_train = 0
        running_loss = 0.0

        print(f"\n🚀 Epoch {epoch+1}/{epochs}")

        for batch_idx, (images, labels) in enumerate(trainloader):

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            outputs = model(images)

            # ✅ UNIVERSAL INCEPTION HANDLER
            if hasattr(outputs, "logits"):  # New PyTorch Inception
                main_output = outputs.logits
                aux_output = outputs.aux_logits
                loss = criterion(main_output, labels)

                if aux_output is not None:
                    loss += 0.4 * criterion(aux_output, labels)

                final_output = main_output

            elif isinstance(outputs, tuple):  # Older versions
                main_output, aux_output = outputs
                loss = criterion(main_output, labels) + \
                       0.4 * criterion(aux_output, labels)
                final_output = main_output

            else:  # Normal models
                loss = criterion(outputs, labels)
                final_output = outputs

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(final_output, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            running_accuracy = 100 * correct_train / total_train

            sys.stdout.write(
                f"\rBatch [{batch_idx+1}/{total_batches}] "
                f"Running Train Acc: {running_accuracy:.2f}%"
            )
            sys.stdout.flush()

        train_accuracy = 100 * correct_train / total_train

        # 🔎 Evaluation
        model.eval()
        correct_test = 0
        total_test = 0

        with torch.no_grad():
            for images, labels in testloader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)

                # Handle Inception outputs
                if hasattr(outputs, "logits"):
                    outputs = outputs.logits
                elif isinstance(outputs, tuple):
                    outputs = outputs[0]

                _, predicted = torch.max(outputs, 1)

                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_accuracy = 100 * correct_test / total_test

        print(f"\n✅ Epoch {epoch+1} Complete | "
              f"Train Acc: {train_accuracy:.2f}% | "
              f"Test Acc: {test_accuracy:.2f}%")

    print("\nTraining Finished 🚀")

    del model
    gc.collect()
    torch.cuda.empty_cache()
    print("GPU cache cleared ✅")


In [42]:
class AlexNet_CIFAR(nn.Module):
    def __init__(self):
        super(AlexNet_CIFAR, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # 32 -> 16

            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # 16 -> 8

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)   # 8 -> 4
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Initialize model
model = AlexNet_CIFAR().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(10):
    model.train()
    running_loss = 0.0
    
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/10], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training Finished")


Epoch [1/10], Loss: 1253.8102, Accuracy: 55.36%
Epoch [2/10], Loss: 938.0080, Accuracy: 62.43%
Epoch [3/10], Loss: 800.3549, Accuracy: 68.11%
Epoch [4/10], Loss: 717.8503, Accuracy: 69.62%
Epoch [5/10], Loss: 647.4908, Accuracy: 70.87%
Epoch [6/10], Loss: 596.8149, Accuracy: 71.46%
Epoch [7/10], Loss: 555.8709, Accuracy: 73.69%
Epoch [8/10], Loss: 513.0678, Accuracy: 74.27%
Epoch [9/10], Loss: 484.2147, Accuracy: 74.74%
Epoch [10/10], Loss: 452.7208, Accuracy: 76.40%
Training Finished


In [53]:
class VGG16_CIFAR(nn.Module):
    def __init__(self):
        super(VGG16_CIFAR, self).__init__()
        
        # Load pretrained VGG16
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        
        # 🔥 Freeze convolution layers (SPEED BOOST)
        for param in self.vgg.features.parameters():
            param.requires_grad = False
        
        # Replace final layer for 10 classes
        self.vgg.classifier[6] = nn.Linear(4096, 10)

    def forward(self, x):
        return self.vgg(x)



# Initialize model
vgg_model = VGG16_CIFAR()

# Train (uses your existing train_model function + loaders)
vgg_acc = train_model(vgg_model, trainloader, testloader, 5)



Epoch [1/5] Batch [0/782] Loss: 2.5525
Epoch [1/5] Batch [100/782] Loss: 1.5754
Epoch [1/5] Batch [200/782] Loss: 1.5595
Epoch [1/5] Batch [300/782] Loss: 1.2969
Epoch [1/5] Batch [400/782] Loss: 1.8074
Epoch [1/5] Batch [500/782] Loss: 1.6756
Epoch [1/5] Batch [600/782] Loss: 1.3521
Epoch [1/5] Batch [700/782] Loss: 1.3890
✅ Epoch [1/5] Final Loss: 1204.5644 Test Accuracy: 59.73%

Epoch [2/5] Batch [0/782] Loss: 1.6999
Epoch [2/5] Batch [100/782] Loss: 1.4707
Epoch [2/5] Batch [200/782] Loss: 1.5450
Epoch [2/5] Batch [300/782] Loss: 1.5055
Epoch [2/5] Batch [400/782] Loss: 1.7289
Epoch [2/5] Batch [500/782] Loss: 1.1932
Epoch [2/5] Batch [600/782] Loss: 1.1339
Epoch [2/5] Batch [700/782] Loss: 1.6635
✅ Epoch [2/5] Final Loss: 1116.9807 Test Accuracy: 60.96%

Epoch [3/5] Batch [0/782] Loss: 1.4482
Epoch [3/5] Batch [100/782] Loss: 1.1631
Epoch [3/5] Batch [200/782] Loss: 1.2441
Epoch [3/5] Batch [300/782] Loss: 1.5925
Epoch [3/5] Batch [400/782] Loss: 1.1870
Epoch [3/5] Batch [500/782]

In [68]:
torch.cuda.empty_cache()

In [57]:
class ResNet50_CIFAR(nn.Module):
    def __init__(self):
        super(ResNet50_CIFAR, self).__init__()
        
        # Load pretrained ResNet50
        self.resnet = models.resnet50(
            weights=models.ResNet50_Weights.IMAGENET1K_V1
        )
        
        # 🔥 Freeze ALL layers first
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # ✅ Unfreeze last residual block (layer4)
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
        
        # Replace final fully connected layer
        self.resnet.fc = nn.Linear(2048, 10)

    def forward(self, x):
        return self.resnet(x)

resnet_model = ResNet50_CIFAR()
resnet_acc = train_model(resnet_model, trainloader, testloader, 5)


Epoch [1/5] Train Loss: 774.2333 Train Acc: 66.62% Test Acc: 73.20%
Epoch [2/5] Train Loss: 512.7203 Train Acc: 77.63% Test Acc: 73.92%
Epoch [3/5] Train Loss: 384.1874 Train Acc: 83.05% Test Acc: 74.38%
Epoch [4/5] Train Loss: 301.9743 Train Acc: 86.88% Test Acc: 74.25%
Epoch [5/5] Train Loss: 246.1540 Train Acc: 89.86% Test Acc: 73.42%
Training Finished 🚀


In [59]:
class ResNet101_CIFAR(nn.Module):
    def __init__(self):
        super().__init__()

        self.resnet = models.resnet101(
            weights=models.ResNet101_Weights.IMAGENET1K_V1
        )

        # Freeze everything
        for param in self.resnet.parameters():
            param.requires_grad = False

        # Unfreeze layer4
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        # Unfreeze BatchNorm layers (important!)
        for m in self.resnet.modules():
            if isinstance(m, nn.BatchNorm2d):
                for param in m.parameters():
                    param.requires_grad = True

        self.resnet.fc = nn.Linear(2048, 10)

    def forward(self, x):
        return self.resnet(x)

resnet101_model = ResNet101_CIFAR()

resnet101_acc = train_model(
    resnet101_model,
    trainloader,
    testloader,
    epochs=8,
    lr=0.0003
)


Epoch [1/8] Train Loss: 769.2146 Train Acc: 65.65% Test Acc: 75.71%
Epoch [2/8] Train Loss: 446.9526 Train Acc: 80.25% Test Acc: 78.38%
Epoch [3/8] Train Loss: 304.4408 Train Acc: 86.56% Test Acc: 79.25%
Epoch [4/8] Train Loss: 207.5968 Train Acc: 90.92% Test Acc: 78.70%
Epoch [5/8] Train Loss: 147.8374 Train Acc: 93.42% Test Acc: 78.50%
Epoch [6/8] Train Loss: 116.6685 Train Acc: 94.80% Test Acc: 79.34%
Epoch [7/8] Train Loss: 95.5177 Train Acc: 95.88% Test Acc: 79.94%
Epoch [8/8] Train Loss: 78.0274 Train Acc: 96.66% Test Acc: 79.39%
Training Finished 🚀


In [76]:
import torch.nn.functional as F
import sys

class EfficientNet_CIFAR(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )

        # Freeze feature extractor
        for param in self.model.features.parameters():
            param.requires_grad = False

        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, 10)

    def forward(self, x):
        x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        return self.model(x)



# Initialize model
efficient_model = EfficientNet_CIFAR()

# Train using your existing function
efficient_acc = train_model(efficient_model, trainloader, testloader, epochs=7)


🚀 Epoch 1/7
Batch [782/782] Running Train Acc: 71.79%
✅ Epoch 1 Complete | Train Acc: 71.79% | Test Acc: 78.83%

🚀 Epoch 2/7
Batch [782/782] Running Train Acc: 76.52%
✅ Epoch 2 Complete | Train Acc: 76.52% | Test Acc: 79.91%

🚀 Epoch 3/7
Batch [782/782] Running Train Acc: 77.14%
✅ Epoch 3 Complete | Train Acc: 77.14% | Test Acc: 80.91%

🚀 Epoch 4/7
Batch [782/782] Running Train Acc: 77.48%
✅ Epoch 4 Complete | Train Acc: 77.48% | Test Acc: 81.03%

🚀 Epoch 5/7
Batch [782/782] Running Train Acc: 77.86%
✅ Epoch 5 Complete | Train Acc: 77.86% | Test Acc: 81.12%

🚀 Epoch 6/7
Batch [782/782] Running Train Acc: 77.93%
✅ Epoch 6 Complete | Train Acc: 77.93% | Test Acc: 81.25%

🚀 Epoch 7/7
Batch [782/782] Running Train Acc: 77.90%
✅ Epoch 7 Complete | Train Acc: 77.90% | Test Acc: 81.20%

Training Finished 🚀
GPU cache cleared ✅


In [8]:
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class InceptionV3_CIFAR(nn.Module):
    def __init__(self):
        super().__init__()

        # Load pretrained Inception v3
        self.model = models.inception_v3(
            weights=models.Inception_V3_Weights.IMAGENET1K_V1,
            aux_logits=True   # IMPORTANT
        )

        # Freeze feature extractor
        for param in self.model.parameters():
            param.requires_grad = False

        # Replace final FC layer
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, 10)

        # Replace auxiliary classifier
        if self.model.aux_logits:
            aux_in = self.model.AuxLogits.fc.in_features
            self.model.AuxLogits.fc = nn.Linear(aux_in, 10)

    def forward(self, x):
        # Resize CIFAR (32x32) → 299x299
        x = F.interpolate(x, size=(299, 299), mode='bilinear', align_corners=False)

        if self.training:
            outputs, aux_outputs = self.model(x)
            return outputs, aux_outputs
        else:
            return self.model(x)


In [9]:
inception_model = InceptionV3_CIFAR()
inception_acc = train_model(inception_model, trainloader, testloader, epochs=7)




🚀 Epoch 1/7
Batch [782/782] Running Train Acc: 66.00%
✅ Epoch 1 Complete | Train Acc: 66.00% | Test Acc: 75.49%

🚀 Epoch 2/7
Batch [782/782] Running Train Acc: 70.72%
✅ Epoch 2 Complete | Train Acc: 70.72% | Test Acc: 75.59%

🚀 Epoch 3/7
Batch [782/782] Running Train Acc: 71.11%
✅ Epoch 3 Complete | Train Acc: 71.11% | Test Acc: 75.94%

🚀 Epoch 4/7
Batch [782/782] Running Train Acc: 71.22%
✅ Epoch 4 Complete | Train Acc: 71.22% | Test Acc: 76.62%

🚀 Epoch 5/7
Batch [782/782] Running Train Acc: 71.48%
✅ Epoch 5 Complete | Train Acc: 71.48% | Test Acc: 76.58%

🚀 Epoch 6/7
Batch [782/782] Running Train Acc: 71.46%
✅ Epoch 6 Complete | Train Acc: 71.46% | Test Acc: 77.32%

🚀 Epoch 7/7
Batch [782/782] Running Train Acc: 71.26%
✅ Epoch 7 Complete | Train Acc: 71.26% | Test Acc: 76.28%

Training Finished 🚀


NameError: name 'gc' is not defined